# Extracted

In [ ]:
!pip install -q vietocr
!pip install -q paddlepaddle
!pip install -q paddleocr

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from paddleocr import PaddleOCR

In [ ]:
config = Cfg.load_config_from_name('vgg_seq2seq')
config['predictor']['beamsearch']=False
config['device'] = 'cuda'
config['weights'] = 'vietocr/transformerocr.pth'
detector = Predictor(config)

In [ ]:
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory

In [ ]:
def image_to_text(image):
    import cv2
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Chuyển sang ảnh đen trắng
    image = cv2.GaussianBlur(image, (3, 3), 0)       # Làm mờ để giảm nhiễu
    # Detect line
    result = ocr.ocr(image, rec=False)

    # Get bbox
    xs = []
    ys = []
    ws = []
    hs = []
    list_text = []

    image = Image.fromarray(image)
    # Create df
    for line in result:
        # Kiểm tra nếu `line` chứa tọa độ hộp bao
        if line is None or len(line) < 1:
            continue
        for bbox_points in line:
        # Giả sử `line` chứa danh sách các tọa độ tại `line[0]`
            try:
                x_coords = [point[0] for point in bbox_points]
                y_coords = [point[1] for point in bbox_points]
                x1, y1 = min(x_coords), min(y_coords)
                x3, y3 = max(x_coords), max(y_coords)
                bbox = (x1 - 10, y1 - 10, x3 + 10, y3 + 10)
            except (IndexError, TypeError):
                continue

            xs.append(bbox[0])
            ys.append(bbox[1])
            ws.append(bbox[2])
            hs.append(bbox[3])
            new_image = image.crop(bbox)
            list_text.append(detector.predict(new_image))

    bbox_df = pd.DataFrame({'x': xs, 'y': ys, 'w': ws, 'h': hs, 'text': list_text})
    bbox_df_reversed = bbox_df[bbox_df['text'].str.len() >= 4].iloc[::-1].reset_index(drop=True)
    text_combined = '\n'.join(bbox_df_reversed['text'])
    return text_combined

In [ ]:
import numpy as np
def extract_text_from_image(image_path):
    # Mở ảnh từ đường dẫn
    image = Image.open(image_path)

    image = np.array(image)

    extracted_text = image_to_text(image)
    if not extracted_text.strip():
        return None
    return extracted_text